### Pistachio - Regression Models

In [ ]:
# Import necessary libraries

# Data visualization
import matplotlib.pyplot as plt

# Data handling (arrays, matrices)
import numpy as np
import pandas as pd
import seaborn as sns

RANDOM_SEED = 300721

In [ ]:
# Reading the data
data = pd.read_excel('./dataset/Pistachio_16_Features_Dataset.xls')

#### Finding the perfect feature to predict:
This dataset is filled with columns that have a direct correlation with each other. This, inevitably leads to good results if those are not removed. So, in order to avoid removing some tables (ranging from 3 to 5) we will instead choose the feature with less correlation with other columns in the dataset.

To do this, we employ a correlation matrix between each and every attribute.

In [ ]:
# Creating a new DataFrame 'df_corr' by dropping the 'Class' column
# Mapping class columns: 'Siit_Pistachio' to 0 and 'Kirmizi_Pistachio' to 1

df_corr = data.drop(columns=['Class'])
df_corr["Is_Siit"] = data["Class"].map({"Siit_Pistachio": 0, "Kirmizi_Pistachio": 1})

corr_matrix = df_corr.corr(numeric_only=False)

f, ax = plt.subplots(figsize=(32, 32))
sns.heatmap(corr_matrix, vmin=-1, square=True, annot=True)

As we can see on the above correlation heatmap, the feature '**EXTENT**' does not have any correlation above 66%, and thus is the chosen one to perform the prediction.

In [ ]:
# Assuming 'Area' is the target variable to predict and other features are used as predictors
predictors = data.drop(['EXTENT', 'Class'], axis=1)  # Drop 'Area' and 'Class' columns
target = data['EXTENT'] 

Now, we can perform some simple data analysis, in specific, relative to missing values and outliers. A deeper analysis is done on the classification problem. 

In [ ]:
# Do predictors have missing values ?
predictors.isnull().any()

In [ ]:
# Do predictors have duplicated values ?
duplicate_count = predictors.duplicated().sum()
display(duplicate_count)

In [ ]:
# Do predictors have outliers ?
numeric_columns = predictors.select_dtypes(include='number').columns

# Create subplots for each numeric column
fig, axes = plt.subplots(nrows=1, ncols=len(numeric_columns), figsize=(30, 5))

# Iterate over numeric columns and create boxplots
for i, column in enumerate(numeric_columns):
    sns.boxplot(x=predictors[column], ax=axes[i], flierprops=dict(markerfacecolor='r', marker='D'))
    axes[i].set_title(f'Boxplot for {column}')
    axes[i].set_ylabel('Value')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# Handling the outliers, by setting them as their closests allowed values
from scipy import stats

# Define a function to replace outliers with the nearest accepted value
def replace_outliers_with_nearest_accepted_value(series):
    # Calculate z-scores for each data point in the series
    z_scores = np.abs(stats.zscore(series))
    
    # Identify outliers using a z-score threshold (3 in this case, but adjustable)
    outliers = z_scores > 3
    
    # Extract non-outliers from the series
    non_outliers = series[~outliers]
    
    # Define a function to find the nearest accepted value
    def find_nearest_accepted_value(x):
        return min(non_outliers, key=lambda v: abs(v - x))
    
    # Replace outliers with the nearest accepted value
    series[outliers] = series[outliers].apply(find_nearest_accepted_value)
    return series

# Apply the replace_outliers_with_nearest_accepted_value function to every numeric column in the DataFrame
for column in predictors.select_dtypes(include=[np.number]).columns:
    predictors[column] = replace_outliers_with_nearest_accepted_value(predictors[column])

In [ ]:
# Converting float64 features to float32 features
for column in predictors.columns:
    if predictors[column].dtype == 'float64':
        predictors[column] = predictors[column].astype('float32')

#### Linear Regression

For the first regression model, we are using a simple Linear Regression.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error

# Splitting the data into training and testing sets (80% train, 20% test)
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.2, random_state=RANDOM_SEED)

# Initializing and training a Linear Regression model
regression_model = LinearRegression(fit_intercept=True)
regression_model.fit(predictors_train, target_train)

# Making predictions on the test set
target_pred = regression_model.predict(predictors_test)

# Evaluating the model using metrics: MAE, MSE, RMSE
mae = mean_absolute_error(target_test, target_pred)
mse = mean_squared_error(target_test, target_pred)
rmse = mean_squared_error(target_test, target_pred, squared=False)
mape = np.mean(np.abs((target_test - target_pred) / target_pred)) * 100

# Displaying the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae:.4f} ({mape:.4f}%)")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


As we can see, we get great results out of the box, there's no need for hyperparameter tunning since linear regression doesn't really have any hyperparameters. 

4% as the error is great, but this value might be due to the correlations of the EXTENT with other features (such as the PERIMETER, SOLIDITY and SHAPEFACTOR_2), but after removing the columns with most correlation factor (>40%) the results obtained were the same within a range of .2%.

#### Gradient Boosting (XDGBoost)

As for the second model, we are using an ensamble learning model, the XDGRegessor, while using grid search for hyperparameters tuning.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Split the data into training and testing sets
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.2, random_state=RANDOM_SEED)

# Define the parameter grid to search through
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5]
}

# Initialize the XGBoost regressor
xgb_model = XGBRegressor(random_state=RANDOM_SEED)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    verbose=1, 
    n_jobs=-1
)

# Fit the grid search to the data
grid_search.fit(predictors_train, target_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_xgb = grid_search.best_estimator_

# Predict on the test set using the best estimator
target_pred = best_xgb.predict(predictors_test)

# Calculate evaluation metrics
mse = mean_squared_error(target_test, target_pred)
r2 = r2_score(target_test, target_pred)

# Evaluating the model using metrics: MAE, MSE, RMSE
mae = mean_absolute_error(target_test, target_pred)
mse = mean_squared_error(target_test, target_pred)
rmse = mean_squared_error(target_test, target_pred, squared=False)
mape = np.mean(np.abs((target_test - target_pred) / target_pred)) * 100

# Displaying the evaluation metrics
print(f"Best Parameters: {best_params}")
print(f"Mean Absolute Error (MAE): {mae:.4f} ({mape:.4f}%)")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Once again, we get a small error (4.18%), pretty similar to the error obtained with linear regression.

#### Result Analysis

For both employed models, the error obtained is not very significant, this is due to the 'EXTENT' feature being obtained from a mathematical formula envolving other columns, this happens throughout a lot of other columns, and explains the great results. W

As already stated above, removing the 'dependent' columns doesn't appear to affect the results, it will lead to, though, the loss of important and needed information.